In [12]:
import pandas as p              # pandas
import os                       # for reading local files and folders
import zipfile                  # for handling zip files
from collections import Counter # for stats analysis on the dataframe  
import math                     # for math functions
import matplotlib.pyplot as plt #
import statistics as stats      #
import folium                   # map

# constants
FOLDER_PATH = "../dataset/2019_01/"
LAT_MIN = 18.76651
LAT_MAX = 22.63089
LON_MIN = -160.11085
LON_MAX = -154.38957

Extract only the data with the correct latitude and longitude values</br>
This operation takes roughly 15-20 seconds per csv file (tested on csv files with > 1 million rows)</br>

In [ ]:
final_df = p.DataFrame()

# list all the files in the specified directory
filenames = os.listdir(FOLDER_PATH)
# and sort them alphabetically
filenames.sort()
# for each file
for file_name in filenames:
    # only iterate over zip files
    if file_name.endswith('.zip'):
        file_path = os.path.join(FOLDER_PATH, file_name)
        # if the csv file has yet to be extracted
        csv_filename = file_name.replace(".zip", ".csv")
        if not(os.path.exists(os.path.join(FOLDER_PATH, csv_filename))):
            # open the zip file and extract it
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                csv_filename = zip_file.namelist()[0]
                zip_file.extractall(FOLDER_PATH)
        # create a dataframe using the extracted csv file
        full_df = p.read_csv(os.path.join(FOLDER_PATH, csv_filename))
        # extract only the rows of interest (with LAT and LON values between the specified bounds)
        filtered_df = full_df[
                                (full_df['LAT'].between(LAT_MIN, LAT_MAX)) &
                                (full_df['LON'].between(LON_MIN, LON_MAX)) 
                            ]
        # and append the extracted rows to the final dataframe
        final_df = p.concat([final_df, filtered_df], ignore_index = True)

print(f"Final dataframe len: {final_df.shape[0]}")
print(final_df.head())
      
    

Separate code snippet used for exporting the final dataframe as a csv file.</br>
This is necessary as the above code snippet takes ~8 minutes read and filter 31 csv files.</br>
By exporting the resulting df as a csv file it's possible import it again for the following part of this notebook.</br>

In [ ]:
#OUTPUT_PATH = os.getcwd() # returns current workink dir
OUTPUT_PATH = FOLDER_PATH
FILE_NAME = '2019_01.csv' #specify file name (.csv extension necessary)

# export the full, uncleaned csv for backup purposes
final_df.to_csv(OUTPUT_PATH + '/' + FILE_NAME, index = False)

print(f"Dataframe shape: {final_df.shape}")

Read the csv file and use it as the final data source</br>
(The code snippet below should be used only if the code snippet #2 was not executed)

In [ ]:
# read csv
final_df = p.read_csv(OUTPUT_PATH + FILE_NAME)
final_df.drop_duplicates(keep='first', inplace=True, ignore_index=True) # drop duplicates
final_df.BaseDateTime = p.to_datetime(final_df.BaseDateTime) # Convert DateTime into more readable format

print(f"Dataframe shape: {final_df.shape}")

Some stats about the imported dataframe</br>
In order to make the histogram below, occurrencies are rounded to the top hundred (ex: 33 -> 100, 244 -> 300, etc...)</br>
This is done in order to have an idea about how many ships have negligible amount of records.</br>
In the next code snippet (#10) the threshold is set to 100 and consequently all vessels with less than 100 data points are removed from the dataset</br>

In [ ]:
# Number of entries (number of rows of the dataframe)
num_of_rows = final_df.index.shape[0]
print("Num of rows: " + str(num_of_rows))

# Extract all the unique MMSIs
unique_vessels_ids = p.unique(final_df['MMSI']).tolist()
num_of_vessels = len(unique_vessels_ids)
print("Num of vessels: " + str(num_of_vessels))

# Calculate the exact number of entries for each ship
vessels_entries_dict = dict(Counter(final_df['MMSI'].to_list())) # save it in a dict
vessels_entries = list(vessels_entries_dict.values()) # save just the values in a list
vessels_entries.sort()
rounded_vessels_entries = [math.ceil(x/100) * 100 for x in vessels_entries]

print(f"Avg entries for each vessel {stats.mean(vessels_entries)}")
print(f"Variance: {stats.variance(vessels_entries)}")
print(f"Standard Deviation: {stats.stdev(vessels_entries)}")
print(f"Max entries found for the same ship: {vessels_entries[-1]}")
print(f"Min entries found for the same ship: {vessels_entries[0]}")

n, bins, patches = plt.hist(x=rounded_vessels_entries, bins='auto', color='orange', edgecolor = 'black')


print(f"Final dataframe len: {final_df.shape[0]}, num of vessels: {len(vessels_entries)}")

Remove ships with not enough points

In [ ]:
THRESHOLD = 100

mmsi_to_be_removed = [x for x in vessels_entries_dict.keys() if vessels_entries_dict[x] <= THRESHOLD]
print(f"Final dataframe len: {final_df.shape[0]}, num of vessels: {len(vessels_entries)}, unwanted vessels: {len(mmsi_to_be_removed)}")
clean_df = final_df.copy()

for unwanted_mmsi in mmsi_to_be_removed:
    clean_df = clean_df.drop(clean_df[clean_df['MMSI'] == unwanted_mmsi].index)

clean_df.sort_values(by='BaseDateTime', inplace=True, ignore_index=True)

print(f"Cleaned dataframe len: {clean_df.shape[0]}, num of vessels: {len(p.unique(clean_df['MMSI']))}")

Export the cleaned dataframe in order to have a backup copy in a csv file

In [ ]:
#OUTPUT_PATH = os.getcwd()
OUTPUT_PATH = FOLDER_PATH
FILE_NAME = 'cleaned_2019_01.csv' #specify file name (don't forget the .csv extension)
clean_df.to_csv(OUTPUT_PATH + '/' + FILE_NAME, index = False)

print(f"Dataframe shape: {final_df.shape}")

Create a new histogram using the "cleaned" data from the clean_df

In [ ]:
clean_vessel_entries = list(Counter(clean_df['MMSI'].to_list()).values())

rounded_clean_vessels_entries = [math.ceil(x/100) * 100 for x in clean_vessel_entries]

print(f"Avg entries for each vessel {stats.mean(clean_vessel_entries)}")
print(f"Variance: {stats.variance(clean_vessel_entries)}")
print(f"Standard Deviation: {stats.stdev(clean_vessel_entries)}")
print(f"Max entries found for the same ship: {max(clean_vessel_entries)}")
print(f"Min entries found for the same ship: {min(clean_vessel_entries)}")

n, bins, patches = plt.hist(x=rounded_clean_vessels_entries, bins='auto', color='green', edgecolor = 'black', )

Print a few points on the map just to check that everything works

In [ ]:
map = folium.Map(location=[21, -158], tiles="OpenStreetMap", zoom_start=9)

clean_df.sort_values(by='MMSI', inplace=True)

#test ranges (replace x and y below): [2:474], [474, 1364], [1364,1837], [1837:2285], [2285, 2725]
x = 2
y = 30000
# extract from the dataset latitude and longitude of the values in range [x:y)
loc = [[str(mmsi), lat, lon] for (mmsi, lat, lon) in zip(clean_df.iloc[x:y]['MMSI'], clean_df.iloc[x:y]['LAT'], clean_df.iloc[x:y]['LON'])]

appendDF = p.DataFrame(loc)
appendDF.columns = ['MMSI', 'LAT', 'LON'] # create a temporary df with the extracted mmsi, lat. and long. values

# CircleMarker requires each point to be added one by one
# The temp function below is applied to all points in the tempDF
def tempFunc(row):
    folium.CircleMarker(
        location=[row['LAT'],row['LON']],
        radius=0.5
        ).add_child(folium.Popup(row['MMSI'])).add_to(map)

# Using 'apply' with a custom function saves a bit of time (compared to a simple for-loop)
appendDF.apply(tempFunc, axis=1)

map 